In [ ]:
import os
import traceback
from glob import glob
import toml
import multiprocessing as mp
import threading as td

from tqdm.auto import tqdm
import numpy as np
import cv2

Load `image_read_test.toml`

In [ ]:
with open("image_read_test.toml", mode="r") as f_reader:
    config = toml.load(f_reader)

load_dir_root = config["model"]["history_root"]
model_name    = config["model"]["model_name"]
model_history = config["model"]["history"]

Load `train_config.toml`

In [ ]:
load_dir = os.path.join(load_dir_root, model_name, model_history)
train_config_path = os.path.join(load_dir, r"train_config.toml")

with open(train_config_path, mode="r") as f_reader:
    train_config = toml.load(f_reader)

dataset_root       = os.path.normpath(train_config["dataset"]["root"])
dataset_name       = train_config["dataset"]["name"]
dataset_gen_method = train_config["dataset"]["gen_method"]
dataset_stdev      = train_config["dataset"]["stdev"]
dataset_param_name = train_config["dataset"]["param_name"]

Generate `path_vars`

In [ ]:
dataset_dir = os.path.join(dataset_root, dataset_name, dataset_gen_method, dataset_stdev, dataset_param_name)
cam_result_root = os.path.join(load_dir, "cam_result")

Run

In [ ]:
train_dir_img_paths = glob(os.path.normpath(f"{dataset_dir}/train/*/*/*.tiff"))

test_dir_img_paths = glob(os.path.normpath(f"{dataset_dir}/test/*/*/*.tiff"))

cam_result_img_paths = glob(os.path.normpath(f"{cam_result_root}/*/*/*.tiff"))

print(f'train_dir_img_paths {type(train_dir_img_paths)}: {len(train_dir_img_paths)}')
print(f'test_dir_img_paths {type(test_dir_img_paths)}: {len(test_dir_img_paths)}')
print(f'cam_result_img_paths {type(cam_result_img_paths)}: {len(cam_result_img_paths)}')

In [ ]:
def image_read_test(img_path_list, pbar_desc, result_queue):
    
    for path in tqdm(img_path_list, desc=f"Image Read Test ({pbar_desc}) "):
        
        try:
            img = cv2.imread(path)
            if not isinstance(img, np.ndarray): raise TypeError(f"Failed to read Image: {path}")
        except Exception as e:
            result_queue.put(traceback.format_exc()) # queue

In [ ]:
if __name__=='__main__':
    
    result_queue = mp.Queue()
    
    t_train = td.Thread(target=image_read_test, args=(train_dir_img_paths, "train", result_queue))
    t_test = td.Thread(target=image_read_test, args=(test_dir_img_paths, "test", result_queue))
    t_cam_result = td.Thread(target=image_read_test, args=(cam_result_img_paths, "cam_result", result_queue))
    t_train.start()
    t_test.start()
    t_cam_result.start()
    
    t_train.join()
    t_test.join()
    t_cam_result.join()
    
    for i in range(result_queue.qsize()): print(result_queue.get())